In [67]:
import numpy as np;from scipy import signal;from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
import h5py;import json
import matplotlib.pyplot as plt;import matplotlib.mlab as mlab 
import readligo as rl
%matplotlib qt

## Load LIGO data.

In [68]:
eventname = 'GW150914' 
plottype = "png"

# Read the event properties from a local json file
fnjson = "BBH_events_v3.json"
try:
    events = json.load(open(fnjson,"r"))
except IOError:
    print("Cannot find resource file "+fnjson)

# Extract the parameters for the desired event:
event = events[eventname]
fn_H1 = event['fn_H1']              # File name for H1 data
fn_L1 = event['fn_L1']              # File name for L1 data
fn_template = event['fn_template']  # File name for template waveform
fs = event['fs']                    # Set sampling rate
tevent = event['tevent']            # Set approximate event GPS time
fband = event['fband']              # frequency band for bandpassing signal

try:
    # read in data from H1 and L1, if available:
    strain_H1, time_H1, chan_dict_H1 = rl.loaddata(fn_H1, 'H1')
    strain_L1, time_L1, chan_dict_L1 = rl.loaddata(fn_L1, 'L1')
except:
    print("Cannot find data files!")

In [69]:
# both H1 and L1 will have the same time vector, so:
time = time_H1
# the time sample interval (uniformly sampled!)
dt = time[1] - time[0]

In [70]:
# plot +- deltat seconds around the event:
deltat = 5
# index into the strain time series for this time interval:
indxt = np.where((time >= tevent-deltat) & (time < tevent+deltat)) 
plt.figure()
plt.plot(time[indxt]-tevent,strain_H1[indxt],'r',label='H1 strain')
plt.xlabel('time (s) since '+str(tevent));plt.ylabel('strain')
plt.legend(loc='lower right');plt.title('Advanced LIGO strain data near '+eventname)
plt.show()

## Amplitude Spectral Density

In [71]:
# Max and min frequncies to be plotted for frequency domain representations of data
f_min,f_max = 20,2000

In [72]:
# Compute FT of windowed strain data
l = strain_H1.size
hamming = np.hamming(l) # Hamming window
tukey = signal.tukey(l, alpha=1./8)# Tukey window
ft = dt*np.fft.rfft(strain_H1*tukey)
freqs = np.fft.rfftfreq(l,d=dt)

In [73]:
%matplotlib qt
plt.figure()
plt.loglog(freqs,abs(ft), 'orange',label = "Raw data")
plt.ylabel('ASD (strain/rtHz)');plt.xlabel('Freq (Hz)')
plt.xlim(f_min,f_max)
plt.grid();plt.legend();plt.show()

$\rightarrow$Noise is reduced by applying Welch's method (a standard averaging technique) on the data

In [74]:
# Compute PSD by Welch's method
f,psd = signal.welch(strain_H1,fs,nperseg = 2**int(np.log2(l/8)))

In [75]:
plt.figure()
plt.loglog(freqs,abs(ft), 'orange',label = "Raw data")
plt.loglog(f,np.sqrt(2*l*psd/fs),color = 'blue',label = "Welch's method")
plt.ylabel('ASD (strain/rtHz)'); plt.xlabel('Freq (Hz)')
plt.xlim(f_min,f_max)
plt.grid();plt.legend();plt.show()

# Noise model and Data Whitening

The strain data is "whitened" - (divided by the noise amplitude spectrum model, in the fourier domain). This suppresses the extra noise at low frequencies and at the spectral lines, to better resolve the weak signals in the most sensitive band.

$\rightarrow$ The resulting time series is no longer in units of strain; now in units of "sigmas" away from the mean noise.

In [76]:
# Noise model
noise_amplitude = interp1d(f, np.sqrt(2*l*psd/fs))

In [77]:
# "Whiten" the data by dividing by the noise amplitude
wh_ft = ft/noise_amplitude(freqs)

# Go back to the time domain
strain_wh = dt*np.fft.irfft(wh_ft,n=l)

In [100]:
# Plot the fourier content of the whitened data
plt.figure()
plt.loglog(freqs,dt*np.abs(wh_ft))
#plt.loglog(freqs,np.abs(ft)*dt*1e21)
plt.xlim(1)
plt.show()

In [90]:
# Plot the whitend data in the time domain
plt.figure()
plt.plot(time-tevent,strain_wh,color='r')
plt.show()

# Filtering

Now the signal is not quite yet visible because of low and high frequency noise in regions far outside that of interest.

$\rightarrow$ Apply a Bandpass filter to the remaining signal.

In [85]:
# Use same bandpass filter as the ligo tutorial!
bb, ab = butter(4, [20*2./fs, fband[1]*2./fs], btype='band')
normalization = np.sqrt((fband[1]-fband[0])/(fs/2))
strain_wh_bd = filtfilt(bb, ab, strain_wh) / normalization

In [105]:
plt.figure()
plt.loglog(freqs,dt*np.abs(np.fft.rfft(strain_wh_bd,n=l)))
#plt.loglog(freqs,dt*np.abs(ft)*1e21)
plt.xlim(1)
plt.show()

In [82]:
plt.figure()
plt.plot(time-tevent,hamming*strain_wh_bd, 'r',label="Filtered Strain data")
plt.xlim([-1,1])
plt.xlabel("time");plt.ylabel("Whitened strain (units of noise stdev)")
plt.legend(loc='upper left');plt.grid();plt.show()

# Matched Filter